In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_1samp
from linearmodels import PanelOLS
from scipy.stats.mstats import zscore

In [2]:
# Dataset
df = pd.read_stata('student_test_data.dta')
df.describe()

,pupilid,schoolid,bungoma,tracking,sbm,girl,agetest,etpteacher,lowstream,stream_meanpercentile,...,r2_spellscore,r2_sentscore24,r2_letterscore24,r2_spellscore24,r2_litscore,r2_mathscoreraw,r2_additions_score,r2_substractions_score,r2_multiplications_score,r2_totalscore
count,7.022000e+03,7022.000000,7022.000000,7022.000000,7022.000000,6995.000000,6500.000000,7022.000000,7022.000000,6462.000000,...,5495.000000,5495.000000,5494.000000,5495.000000,5495.000000,5489.000000,5495.000000,5495.000000,5495.000000,5489.000000
mean,7.831203e+06,783.000977,0.241811,0.514526,0.508402,0.495354,9.274769,0.496297,0.257477,50.640678,...,3.703367,5.827371,11.951511,8.888080,9.106643,10.750592,5.112648,4.287352,1.338853,19.864565
std,1.714448e+06,171.442780,0.428211,0.499825,0.499965,0.500014,1.470396,0.500022,0.437275,18.528595,...,3.610104,6.967761,8.223339,8.664250,7.301884,5.204099,2.376443,2.268766,1.433964,11.312814
min,4.301001e+06,430.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,23.811319,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.921029e+06,692.000000,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,26.941175,...,0.000000,0.000000,4.457143,0.000000,2.485714,7.000000,4.000000,3.000000,0.000000,10.700000
50%,7.951019e+06,795.000000,0.000000,1.000000,1.000000,0.000000,9.000000,0.000000,0.000000,51.094471,...,3.000000,3.000000,12.000000,7.200000,6.700000,11.000000,5.000000,5.000000,1.000000,18.514286
75%,9.381056e+06,938.000000,0.000000,1.000000,1.000000,1.000000,10.000000,1.000000,1.000000,73.787064,...,7.000000,9.600000,19.542856,16.799999,16.014286,15.000000,7.000000,6.000000,3.000000,29.700001
max,1.020106e+07,1020.000000,1.000000,1.000000,1.000000,1.000000,19.000000,1.000000,1.000000,76.718063,...,10.000000,24.000000,24.000000,24.000000,24.000000,24.000000,9.000000,9.000000,6.000000,46.099998


In [3]:
df.head()

,pupilid,schoolid,district,bungoma,division,zone,tracking,sbm,girl,agetest,...,r2_spellscore,r2_sentscore24,r2_letterscore24,r2_spellscore24,r2_litscore,r2_mathscoreraw,r2_additions_score,r2_substractions_score,r2_multiplications_score,r2_totalscore
0,4301001,430.0,BUNGOMA,1.0,KANDUYI,MUNICIPALITY,1.0,1.0,0.0,NaN,...,0.0,4.8,0.0,0.0,2.20,5.0,3.0,2.0,0.0,7.20
1,4301002,430.0,BUNGOMA,1.0,KANDUYI,MUNICIPALITY,1.0,1.0,1.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4301003,430.0,BUNGOMA,1.0,KANDUYI,MUNICIPALITY,1.0,1.0,1.0,8.0,...,0.0,4.2,0.0,0.0,1.05,2.0,1.0,1.0,0.0,3.05
3,4301004,430.0,BUNGOMA,1.0,KANDUYI,MUNICIPALITY,1.0,1.0,0.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4301005,430.0,BUNGOMA,1.0,KANDUYI,MUNICIPALITY,1.0,1.0,0.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 7a
df_reduced =  df.dropna(subset = ['schoolid', 'tracking', 'totalscore'])
print(smf.ols(formula = 'zscore(totalscore) ~ C(tracking)', data = df_reduced).fit(cov_type='cluster', cov_kwds={'groups': df_reduced['schoolid']}).summary())

                            OLS Regression Results                            
Dep. Variable:     zscore(totalscore)   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.197
Date:                Tue, 03 Dec 2019   Prob (F-statistic):             0.0763
Time:                        18:41:16   Log-Likelihood:                -8208.9
No. Observations:                5795   AIC:                         1.642e+04
Df Residuals:                    5793   BIC:                         1.644e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0710      0

In [5]:
#7a
df_reduced =  df.dropna(subset = ['schoolid', 'tracking', 'totalscore', 'agetest', 'etpteacher', 'bottomhalf', 'bottomquarter', 'secondquarter', 'topquarter', 'girl', 'percentile'])
print(smf.ols(formula = 'zscore(totalscore) ~ C(tracking) + agetest + C(etpteacher) + C(girl) + percentile + C(bottomhalf) * C(tracking) + C(bottomquarter) * C(tracking) + C(secondquarter) * C(tracking) + C(topquarter) * C(tracking)', data = df_reduced).fit(cov_type='cluster', cov_kwds={'groups': df_reduced['schoolid']}).summary())

                            OLS Regression Results                            
Dep. Variable:     zscore(totalscore)   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     51.85
Date:                Tue, 03 Dec 2019   Prob (F-statistic):           9.65e-41
Time:                        18:41:17   Log-Likelihood:                -6705.8
No. Observations:                5269   AIC:                         1.344e+04
Df Residuals:                    5255   BIC:                         1.353e+04
Df Model:                          13                                         
Covariance Type:              cluster                                         
                                                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------